Experiment: Summing together last 4 attention distribution
------
Summing together the last four attention distributions; using only one attention head.

In [177]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
attention_dataloader = dataset.return_dataloader()
attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/01/2019 16:35:31 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


0it [00:00, ?it/s]

500it [00:00, 6887.20it/s]06/01/2019 16:35:31 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/01/2019 16:35:32 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../../../tasks/bias_classification/resul

In [ ]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

In [156]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [165]:
# Extracting indices of bias labels 
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)
# Summing together bias distributions
summed_dist = sum_attention_dist(biased_tok_dist)

In [166]:
windowed_data, removed_indices = window_attention_dist(summed_dist, bias_indices, window_size=params.final_task['window_size'])
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(windowed_data), "attention_dist")
dataset.shuffle_data()

In [167]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

Beginning Training Procedures

In [168]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [169]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [174]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [180]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.4971744799613953


In [182]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [187]:
print(max_auc)

0.7796687808315715
